<font size=4 color='blue'>

# Clase 3, octubre 7 del 2020


<font size=4 color='blue'>
    
## Generación de las muestras



<font size=5 color='b'>

1.  Se genera un conjunto de numeros aleatorios ($(x_1,y_1),(x_2,y_2),…,(x_n,y_n)$)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Se modelan un conjunto de datos (muestras)
#En una versión futura del código, estos serán sustituidos por datos 
#medidos experimentalmente, y simplemente serán leidos.

def generador_datos_simple(beta, m, desviacion):
    
    np.random.seed(3)                      # se fija para que los valores sean reproducibles
    x = np.random.random(m) * 10           # x es arreglo con m numeros aleatorios entre 0 y 100
    e = np.random.randn(m) * desviacion    # e es un error generado aleatoriamente
    y = x * beta + e                       # se obtienen los valores de y 
                                           # x*beta genera una recta, sumando los errores, e, se alejan los puntos de la recta.     
    return x.reshape((m,1)), y.reshape((m,1))

In [ ]:
desviacion = 10
beta = 4
m = 5000   # numero de muestras

x, y = generador_datos_simple(beta, m, desviacion)
plt.figure(figsize=(13,8))
plt.scatter(x, y)
plt.grid(True)
plt.title('Distancia vs tiempo', size=24)
plt.xlabel('Tiempo (s)', size=18)
plt.ylabel('Distancia (cm)', size=18);

<font size=4 color ='blue'>
Se generan histogramas de los datos originales, tanto de (x) como de (y)

In [ ]:
plt.figure(figsize=(13,5))

plt.subplot(1, 2, 1)
plt.hist(x, bins=30, edgecolor='black', alpha=0.5)
plt.xlabel('tiempo(s)', fontsize=16)
plt.ylabel('frecuencia', fontsize=16)

plt.subplot(1, 2, 2)
plt.hist(y, bins=30, edgecolor='black', alpha=0.5)
plt.xlabel('distancia(cm)', fontsize=16)
plt.ylabel('frecuencia', fontsize=16);

<font size=2, color='blue'>
    
# Buscando la correlación entre las muestras:



<font size=4>

Se tiene un conjunto de muestras (puntos) $(x_i, y_i)$, y se busca encontrar una función F que describa una posible correlación entre ellos. En una primera aproximación x (tiempo) es una variable independiente, mientras que y (distancia) depende de x.

Para encontrar la correlación entre las muestras, proponemos un conjunto de funciones definidas mediante la siguiente relación lineal:

$$ F(x, w, b) = b + w x $$
  
Vemos que esta relación funcional es derivable respecto a todas sus variables, $x, w, b$.


La letra $w$ se emplea como abreviación de la palabra en inglés "weight", porque se relaciona con la importancia que tiene la variable $x$ en el valor de la función $F$. La letra $b$ es la abreviación de la palabra "bias" en inglés, y se refiere a la referencia respecto a cero de la función $F$.


<font size = 4>

Para encontrar la función que describe la correlación entre los puntos, es necesario generar una métrica para describir qué tanto se acerca cada una de las funciones específicas (con valores definidos de $w$ y de $b$) a esta descripción.

La métrica que se propone es la siguiente: $$ $$

Para cada muestra $(x_i, y_i)$ se evalua $F(x_i,w,b)$ y se compara con el correspondiente valor $y_i$, la diferencia entre estos valores se eleva al cuadrado. $$ (y_i-F(x_i,w,b))^{2}$$


Finalmente se calcula el promedio de este valor sobre todas las muestras, el cual definimos como error cuadrático medio (MSE, por sus siglas en inglés, mean squared error). 
Si $m$ es el número de muestras, el MSE queda como:

$$MSE = \dfrac{1}{m} ∑_{i=1}^{m}(y_i-F(x_i,w,b))^{2} $$

<font size=4, color='blue'>
    
En el siguiente código se implementa la generación del error cuadrático medio dada una función específica definida por los pesos iniciales $w = weight_0$, y $b = bias_0$.

<font size =4>
    

Los parámetros $w$ y $b$ se eligen al azar: 

``` python 

weight_0 = random.random()*10  
 
bias_0 = random.random()*10   

```

In [ ]:
np.random.seed(10)   # se usa la semilla para tener valores reproducibles

#Initializing the variables of the function f

weight_0 = np.random.random()*10
bias_0 = np.random.random()*10

print('w_0:', np.round(weight_0, 4), 'bias_0:', np.round(bias_0, 4))

<font size=4 color="blue">

Se grafica la correspondiente función $F(X,w,b)$, junto con los puntos que representan a las muestras

In [ ]:
#1. The following arrays are generated for plotting the Function F(x, weight_0, bias_0)

x_ = np.arange(0.0, 10.0, 0.1) #genera un cto de valores iniciando en 0.0, terminando en 10.0 y a pasos de 0.1
y_ = weight_0*x_ + bias_0      # y es la función F(x_, w, b) generada con w_0 y b_0 

#2. Using this function F, the residuos is calculated by comparin the calculated and measured values

residuo = 0
for i in range(len(x)):   # para cada muestra:
    r = (y[i] - weight_0*x[i] - bias_0)**2
    residuo += np.squeeze(r)
residuo = residuo/len(x)

print('residuo:', residuo)


#3. Samples and function F are plotted

plt.figure(figsize=(13,8))
plt.grid(True)
    #Plotting function
plt.plot(x_, y_, color='green', lw=4, label='F(x, w, b)') # x_, y_ son los valores que definen a F(X, w_0, b_0)
plt.legend()
    #Plotting samples
plt.scatter(x, y);                                        # x, y son las muestras generadas

<font size=5, color='blue'>
    
> 1. Se varian los valores de los parámetros $w$ y $b$ buscando reducir el residuo.

<font size=5, color='blue'>

Se emplea el método de gradiente descendente para realizar esta variación.

<font size=4 color='black'>
    
Los parámetros $w$ y $b$ se deben actualizar, de manera que el MSE disminuya. 

Los parámetros se actualizan usando la siguientes ecuaciones:

$$ w := w - \alpha \dfrac{\partial MSE(w, b)}{\partial w}$$

$$ b := b - \alpha \dfrac{\partial MSE(w, b)}{\partial b}$$

$\alpha$: tasa de aprendizaje, es un hiperparámetro del modelo, y controla la velocidad con que el modelo aprenderá a ajustar a los parametros $w$ y $b$.


En este caso

$$ MSE = \dfrac{1}{m}∑_{i=1}^{m}(y_i-f(x_i))^{2} = \dfrac{1}{m}∑_{i=1}^{m}(y_i - w x_i - b)^2 $$

Entonces

$$ \dfrac{\partial MSE(w, b)}{\partial w} = \dfrac{2}{m}∑_{i=1}^{m}[(w x_i + b -y_i)(x_i)]$$

$$ \dfrac{\partial MSE(w, b)}{\partial b} = \dfrac{2}{m}∑_{i=1}^{m}[(w x_i + b -y_i)]$$



In [ ]:
#Function to update weight and bias

def update_parameters(x, y, weight, bias, alfa, iteraciones):
    
    '''
    Esta función actualiza los parámetros (w,b) usando gradient descent
    
    INPUT
        x,y: muestras
        weight: peso inicial
        bias: bias inicial
        alfa: learning rate
        iteraciones: int que define el numero de veces a actualizar a los parámetros
    OUTPUT
        weights: lista con los pesos actualizados en cada iteración
        biases: lista con los bias actualizados en cada iteración
        residuos: lista con los residuos actualizados en cada iteración'''
    
    #1. inicializacion de parametros
    
    x = np.squeeze(x)
    y = np.squeeze(y)
    alfa = alfa
    residuo = 0
    d_w = 0.0
    d_b = 0.0
    m = len(x)  # especifica el número de muestras

    #2. Especificaciones de las graficas
    
    plt.figure(figsize=(20,8))    

    ax1 = plt.subplot(1,3,1)
    ax2 = plt.subplot(1,3,2)
    ax3 = plt.subplot(1,3,3)

    ax1.grid(True)
    ax2.grid(True)
    ax3.grid(True)
    
    ax1.scatter(x, y) # grafica las muestras
    ax1.set_title('Distancia vs tiempo', size=24)
    ax1.set_xlabel('Tiempo', size=18)
    ax1.set_ylabel('Distancia', size=18)

        # Recta generada con los parametros iniciales

    y_ = weight*x + bias     # es F(x, w, b)
    
    ax1.plot(x, y_, color='green', lw=4)
    
    #3. Se calcula el residuo
    
    weights = []      # se inicializan las listas weights, biases, residuos
    biases = []
    residuos = []
    
    for i in range(iteraciones):  

        # calculo de derivadas y el residuo

        for i in range(m):

            d_w += 2*(weight*x[i]+bias- y[i])*x[i]      
            d_b += 2*(weight*x[i]+bias-y[i])
            residuo += (y[i]-weight*x[i]-bias)**2

        residuo /= m
        d_w /= m
        d_b /= m
        
        weights.append(weight)      # se agregan los resultados calculados
        biases.append(bias)
        residuos.append(residuo)
        
        #4. Actualización de los parametros

        weight = weight - alfa*d_w
        bias = bias - alfa*d_b

        # Recta generada con la actualizacion de los parametros
        
        y_ = weight*x + bias    # es F(x,w_actualizado, b_actualizado)
        ax1.plot(x, y_, lw=4)

        #5. Grafica de los residuos como función de uno de los parámetros (el peso)
        
        ax2.scatter(weight, residuo)
        ax2.set_title('MSE vs weight', size=24)
        ax2.set_xlabel('weight', size=18)
        ax2.set_ylabel('MSE', size=18)

        #6. Grafica de los residuos como función de uno de los parámetros (el bias)
        
        ax3.scatter(bias, residuo)
        ax3.set_title('MSE vs bias', size=24)
        ax3.set_xlabel('bias', size=18)
        ax3.set_ylabel('MSE', size=18)

    return weights, biases, residuos

In [ ]:
weight = weight_0 
bias = bias_0 
alfa = 0.001
num_iter = 100

weights_100, biases_100, residuos_100 = update_parameters(x, y, weight, bias, alfa, num_iter)

<font size=4, color='blue'>
Se grafica el residuo como función de cada iteración en que se modificaron el bias y el peso.

In [ ]:
plt.figure(figsize=(13, 8))
plt.grid(True)
plt.scatter(range(num_iter), residuos_100, color='blue')
plt.title('MSE vs Iteración', size=20)
plt.xlabel('Iteración', size=15)
plt.ylabel('MSE', size=15);

<font size=5, color='blue'>

> 2. Evaluación del ajuste obtenido


<font size=4, color='blue'>
El total de los datos son divididos en dos grupos: uno con el 90 % de los datos y el segundo con el restante 10 %

<font size=4>
    
La funcion shuffle reordena de forma aleatoria la posición de un conjunto de datos en una lista.

     x = [ 1, 5, 7, 3, 8]
     shuffle(x) = [5, 8, 3, 1, 7]

 La funcion zip permite hacer conjuntos ordenados de datos combinando dos vectores de igual dimensión, por ejemplo

     x = [1, 2, 3]
     y = [5, 6, 7]
    
     zip(x, y) = ((1, 5), (2, 6), (3, 7))
    
 De esta manera, junto con la funcion shuffle, se asegura que los datos correspondientes a $x$ y $y$ intercambian su posición de la misma manera.
 
 Por otra parte, la operación zip(*c)
 
     (x, y) = zip(*c)
     
Separa a los datos $x$ e $y$ ya mezclados.

In [ ]:
#1. Los datos se cambian de posición aleatoriamente

from random import shuffle

c = list(zip(x, y))    # se juntan las muestras: x e y ---> (x,y)
shuffle(c)             # se cambia el orden de las muestras (x,y)
(x, y) = zip(*c)       # se separan las muestras: (x,y)---> x e y

print('Se tienen', len(x), 'muestras en total')

#2. Los datos se dividen

samples_for_train = (x[0:int(0.90*len(x))], y[0:int(0.90*len(y))])
print('Se van a usar', len(samples_for_train[0]), 'muestras para el ajuste')

samples_for_test = (x[int(0.90*len(x)):], y[int(0.90*len(y)):])
print('Se van a usar', len(samples_for_test[0]), 'muestras para probar')

#3. Se grafican ambos conjuntos

plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 1)
plt.grid(True)
plt.scatter(samples_for_train[0], samples_for_train[1])
plt.title('samples_for_train', size=20)
plt.xlabel('Tiempo(s)', size =15)
plt.ylabel('Distancia(cm)', size =15)

plt.subplot(1, 2, 2)
plt.grid(True)
plt.scatter(samples_for_test[0], samples_for_test[1], color='red')
plt.title('samples_for_test', size=20)
plt.xlabel('Tiempo(s)', size =15)
plt.ylabel('Distancia(cm)', size =15);

<font size =4, color = 'blue'>
    
 ## Normalización de las muestras que se emplearan para el ajuste
 

<font size=4 color='b'>
    
 Si se normalizan con 3 veces la desviacion estandar, el 99.7% de los datos tendrán valores entre -1 y 1. El rango con que se normaliza tambien se puede variar entre una desviación o dos desviaciones estandar. En el primer caso el 68 % de los datos tendrán valores entre -1 y 1, mientras que en el segundo caso este rango correspondera al 95 % de los datos.

<font size=5, color='blue'>
Normalizando con una desviación estandar

<font size=4>
    
 1. Separación de muestras

In [ ]:
x_ajustar = samples_for_train[0]     # samples_for_train es de la forma (x,y)
y_ajustar = samples_for_train[1]     # aqui se separan: (x,y) ---> x e y

<font size=4>
    
 2. Normalización de muestras

In [ ]:
y_1 = (y_ajustar-np.mean(y_ajustar))/(1.0*np.std(y_ajustar))    
x_1 = (x_ajustar-np.mean(x_ajustar))/(1.0*np.std(x_ajustar))

<font size=4>
    
 3. Histogramas de las muestras

In [ ]:
plt.figure(figsize=(13,5))

plt.subplot(1, 2, 1)
plt.grid(True)
plt.hist(x_1, bins=30,color='red', edgecolor='b', alpha=0.5)
plt.xlabel('tiempo (unidades relativas)', fontsize=14)
plt.ylabel('frecuencia',fontsize=14)

plt.subplot(1, 2, 2)
plt.grid(True)
plt.hist(y_1, bins=30,color='red', edgecolor='b', alpha=0.5)
plt.xlabel('distancia (unidades relativas)',fontsize=14)
plt.ylabel('frecuencia',fontsize=14);

<font size=5, color='blue'>
Normalizando con tres desviaciones estandar

<font size=4>
    
 1. Normalización de muestras

In [ ]:
y_3 = (y_ajustar-np.mean(y_ajustar))/(3.0*np.std(y_ajustar))
x_3 = (x_ajustar-np.mean(x_ajustar))/(3.0*np.std(x_ajustar))

<font size=4>
    
 2. Histogramas de las muestras

In [ ]:
plt.figure(figsize=(13,5))

plt.subplot(1, 2, 1)
plt.grid(True)
plt.hist(x_3, bins=30,color='red', edgecolor='b', alpha=0.5)
plt.xlabel('tiempo(unidades relativas)',fontsize=14)
plt.ylabel('frecuencia',fontsize=14)

plt.subplot(1, 2, 2)
plt.grid(True)
plt.hist(y_3, bins=30,color='red', edgecolor='b', alpha=0.5)
plt.xlabel('distancia(unidades relativas)',fontsize=14)
plt.ylabel('frecuencia',fontsize=14);

<font size=5, color='blue'>
Normalizando con dos desviaciones estandar

<font size=4>
    
 1. Normalización de muestras

In [ ]:
y_2 = (y_ajustar-np.mean(y_ajustar))/(2.0*np.std(y_ajustar))
x_2 = (x_ajustar-np.mean(x_ajustar))/(2.0*np.std(x_ajustar))

<font size=4>
    
 2. Histogramas de las muestras

In [ ]:
plt.figure(figsize=(13,5))

plt.subplot(1, 2, 1)
plt.grid(True)
plt.hist(x_2,bins=30, color='red', edgecolor='b', alpha=0.5)
plt.xlabel('tiempo(unidades relativas)', fontsize=14)
plt.ylabel('frecuencia',fontsize=14)

plt.subplot(1, 2, 2)
plt.grid(True)
plt.hist(y_2, bins=30,color='red', edgecolor='b', alpha=0.5)
plt.xlabel('distancia(unidades relativas)',fontsize=14)
plt.ylabel('frecuencia',fontsize=14);

<font size=5, color='blue'>
Trabajaremos entonces normalizando los datos con una desviación estandar

In [ ]:
samples_train_x = samples_for_train[0]     # samples_for_train es de la forma (x,y)
samples_train_y = samples_for_train[1]     # aqui se separan: (x,y) ---> x e y

In [ ]:
mean_distance = np.mean(samples_train_y)
std_distance = np.std(samples_train_y)

samples_for_train_y = (samples_train_y-mean_distance)/std_distance

mean_time = np.mean(samples_train_x)
std_time = np.std(samples_train_x)

samples_for_train_x = (samples_train_x-mean_time)/std_time

In [ ]:
plt.figure(figsize=(13,5))

plt.subplot(1, 2, 1)
plt.grid(True)
plt.hist(samples_for_train_x, bins=30,color='blue', edgecolor='b', alpha=0.5)
plt.xlabel('Tiempo(unidades relativas)', fontsize=14)
plt.ylabel('Frecuencia',fontsize=14)

plt.subplot(1, 2, 2)
plt.grid(True)
plt.hist(samples_for_train_y, bins=30,color='blue', edgecolor='b', alpha=0.5)
plt.xlabel('Distancia(unidades relativas)',fontsize=14)
plt.ylabel('Frecuencia',fontsize=14);

<font size=4, color='blue'>
Se vuelve a correr el ajuste de datos empleando el metodo de gradiente descendente, pero usando sólo el conjunto de datos correspondiente al 90 %

In [ ]:
#np.random.seed(3)
weight_0 = np.random.random()
bias_0 = np.random.random()
alfa = 0.04
num_iter = 100

weights_train, biases_train, residuos_train = update_parameters(samples_for_train_x, 
                                                                samples_for_train_y, 
                                                                weight_0, bias_0, alfa, num_iter)

<font size=4, color='blue'>
Graficar el error cuadrático medio obtenido como función de la iteración

In [ ]:
plt.figure(figsize=(13, 8))
plt.grid(True)
plt.scatter(range(num_iter), residuos_train, color='blue')
plt.title('MSE vs iteración', size=20)
plt.xlabel('iteración', size=15)
plt.ylabel('MSE', size=15)
plt.legend(['MSE train'],loc=0);

In [ ]:
def residuo(x, y, weight, bias):
    r = 0                                # el residuo se inicializa en 0
    m = len(x)                           # m indica el número de muestras
    for i in range(m):
        r = (y[i]-weight*x[i]-bias)**2
        r += np.squeeze(r)
    r /= m  
    return r

In [ ]:
print("Residuo mas pequeño que se obtuvo durante el ajuste =  %.5f" \
      %np.squeeze(residuo(samples_for_train_x, samples_for_train_y, weights_train[-1], biases_train[-1])))


<font size=4, color='blue'>
Con los valores óptimos del bias y el peso se calcula el error cuadrático medio que se obtiene con el 10 % de los datos restantes (conjunto test).

<font size=4, color='black'>

Para calcular el residuo asociado a las samples_for_test, estas se normalizan con los parámetros (valore medio y desviación estandar) utilizados para normalizar los datos de las samples_for_train.

In [ ]:
y_test = (samples_for_test[1]-mean_distance)/std_distance
x_test = (samples_for_test[0]-mean_time)/std_time

<font size=4, color='blue'>

Con los valores óptimos obtenidos para el peso y el bias, evaluamos ahora el residuo que se obtiene las muestras_test.

In [ ]:
print("Residuo =  %.5f" %np.squeeze(residuo(x_test, y_test, weights_train[-1], biases_train[-1])))

<font size=5, color='blue'>

> 3. Otra forma de evaluar el ajuste obtenido.

<font size=4, color='blue'>
   
En este caso, las muestras seleccionadas para hacer el ajuste se dividen en dos grupos:

El 90 % (este valor es solo un ejemplo) de ellos se emplea para hacer el ajuste (train set),
    
el 10 % restantes se emplean para evaluar el residuo que se obtiene en cada iteración del ajuste (validation set).


<font size=4, color='black'>

Modificamos la función que se tiene para actualizar el peso y se bias en cada ciclo.

A esta nueva función se le proporciona la proporción de los datos que se van a emplear para validar el ajuste.

In [ ]:
#Function to update weight and bias

def update_parameters_1(x, y, weight, bias, alfa, iteraciones, val_ratio=0.1):
    
    '''
    Esta función actualiza los parámetros (w,b) usando gradient descent
    Además separa a x e y en dos conjuntos: train y validation usando el val_ratio
    
    INPUT
        x,y: muestras
        weight: peso inicial
        bias: bias inicial
        alfa: learning rate
        iteraciones: int que define el numero de veces a actualizar a los parámetros
        val_ratio: porcentaje de los datos (x,y) a usar como conjunto de validación
    OUTPUT
        weights: lista con los pesos actualizados en cada iteración
        biases: lista con los bias actualizados en cada iteración
        residuos: lista con los residuos actualizados en cada iteración para el conjunto de entrenamiento
        residuos_val: lista con los residuos actualizados en cada iteración para el conjunto de validación'''
    
    #1.  inicializacion de parametros
    
    x = np.squeeze(x)
    y = np.squeeze(y)
    alfa = alfa
    residuo = 0
    d_w = 0.0
    d_b = 0.0
    m = len(x)

    #2.  Especificaciones de las graficas
    
    plt.figure(figsize=(13,8)) 
    plt.title('MSE vs iteracion', size=24)
    plt.xlabel('iteracion', size=18)
    plt.ylabel('MSE', size=18)
    
    #3. se dividen las muestras en los conjuntos train y validation
    
    ajustar_ratio = int((1.0-val_ratio)*len(x))  
   
    samples_train = (x[0:ajustar_ratio], y[0:ajustar_ratio])
    samples_val = (x[ajustar_ratio:], y[ajustar_ratio:])
    x = samples_train[0]
    y = samples_train[1]
    x_val = samples_val[0]
    y_val = samples_val[1]
    
    #3.1 Normalización
    
    mean_x = np.mean(x)
    std_x = np.std(x)
    
    mean_y = np.mean(y)
    std_y = np.std(y)
    
    x = (x-mean_x)/std_x
    y = (y-mean_y)/std_y
    x_val = (x_val-mean_x)/std_x
    y_val = (y_val-mean_y)/std_y

    #4. se calcula el residuo
       
    weights = []
    biases = []
    residuos = []
    residuos_val = []
    
    m_ajustar = len(x)    # numero de muestras para el ajuste
    m_val = len(x_val)    # numero de muestras para validar
    
    for i in range(iteraciones):

        # calculo de derivadas y el residuo
        residuo = 0.0
        residuo_val = 0.0
        
        for j in range(m_ajustar):

            d_w += 2*(weight*x[j] + bias- y[j])*x[j]
            d_b += 2*(weight*x[j]+bias-y[j])
            residuo += (y[j]-weight*x[j] - bias)**2

        residuo /= m_ajustar
        d_w /= m_ajustar
        d_b /= m_ajustar
        
        #calculo del residuo de las muestras de validación
        
        for j in range(m_val):
            residuo_val += (y_val[j]-weight*x_val[j] - bias)**2
        residuo_val /= m_val
                      
        weights.append(weight)             # se agregan los valores calculados a las listas
        biases.append(bias)
        residuos.append(residuo)
        residuos_val.append(residuo_val)
        
        #5. Actualizacion de los parametros

        weight = weight - alfa*d_w
        bias = bias - alfa*d_b
        
        #6. Graficas del residuo en función de la iteración
        
        plt.grid(True)
        plt.scatter(i, residuo, color='blue')
        plt.scatter(i, residuo_val, color='orange')
    plt.legend(['residuo', 'residuo_val'],loc=0);


    return weights, biases, residuos, residuos_val, mean_x, std_x, mean_y, std_y

In [ ]:
#np.random.seed(2)
weight_0 = np.random.random()
bias_0 = np.random.random()
alfa = 0.04
num_iter = 100
validacion_ratio = 0.5

weights, biases, residuos, residuos_val, mean_x, std_x, mean_y, std_y = update_parameters_1 \
        (samples_for_train[0], samples_for_train[1], weight_0, bias_0, alfa, num_iter, validacion_ratio)


<font size=4, color='blue'>

Con los valores óptimos obtenidos para el peso y el bias, evaluamos ahora el residuo que se obtiene las muestras_test.

In [ ]:
print("Residuo =  %.5f" %np.squeeze(residuo(x_test, y_test, weights[-1], biases[-1])))

<font size=5, color='blue'>

Inferencia

<font size=4, color='black'>

Dado el tiempo t podemos obtener una predicción (inferencia) del valor de la distancia.

In [ ]:
def inference(t, w, b, mean_t, std_t, mean_y, std_y) :
    t = (t-mean_t) / std_t 
    d = w * t + b
    d = d * std_y + mean_y
    return d

In [ ]:
tiempos = [1.65, 2.20, 4.5, 8.7]

for t in tiempos :
    distance = inference(t,weights[-1], biases[-1], mean_x, std_x, mean_y, std_y)

    print ("Para el tiempo de {0:5.3f} s la distancia inferida es {1:6.3f} cm ".format(t,distance))

<font size=4 color='black'>

Vemos entonces que con el ajuste de una función al conjunto de puntos $(x_i, y_i)$ podemos hacer predicciones de valores y dado el valor de $x_i$. 
    
Es decir, el sistema desarrollado "aprendió" la correlación que hay entre las $x_i$ y la $y_i$, y por ello puede hacer inferencias.

<font size=6 color='blue'>

Inteligencia Artificial

<font size=5 color='blue'>

Pasaremos conceptualmente del ajuste de datos a la Inteligencia Artificial

<font size=4 color='black'>

Esta transición la haremos empleando toda la matemática desarrollada en las celdas anteriores

<font size=4 color='black'>
    
En inteligencia artificial, un sistema es inteligente cuando después de ser entrenado con información que le es suministrada, es capaz de hacer inferencias.



<font size=5 color='blue'>
 
Dada la dinámica en inteligencia artificial a nivel mundial, en el presente curso,
    
emplearemos la nomenclatura estandar que se emplea en el idioma Inglés.

<font size=4 color='black'>

Al analizar una area de estudio, se deben encontrar las conceptos que la identifican.
    
Se hace una cuantificación de estos conceptos definiendo variables que identifican a cada uno de ellos.
    
Existe un conjunto que estas variables $\textbf{X}$ que son idependientes y que determinan al resto, a las cuales denominaremos como $\textbf {Y}$.
    



<font size=4 color='black'>

Se obtiene un cojunto $m$ de muestras de estas variables.

Se genera un sistema de aprendizaje, al cual se le suministran estas muestras.

Con esta información el sistema aprende y puede hacer inferencias.

Por ejemplo, si se le presenta un nuevo dato X del área de estudio, el sistema puede predecir (inferir) los correspondientes valores de las variable Y.

<font size=5 color='blue'>
    
El ejemplo que vimos de ajuste de una función a un conjunto de puntos $(x_i, y_i)$ lo traduciremos a un sistema de aprendizaje artificial.

<font size=4 color='black'>
    
El área que analizamos corresponde al movimiento de un cuerpo. 
    
Esta área la caracterizamos por los conceptos tiempo y distancia. 
    
Definimos la viable $x$ para identificar al tiempo y la variable $y$ para idenficar la distancia.

<font size=5 color='blue'>

Analizaremos un ejemplo de esta área.

<font size=4 color='black'>

Tenemos un cuerpo, del cual se han obtenido $m$ muestras de estas variables. Cada muestra de este cuerpo la identificamos por la dupla $(x_i, y_i)$.
    
Generaremos un sistema de aprendizaje, en donde proponemos que la función $F(x)$ describe la relación entre estas variables. 
    
Esta función puede tener diferentes formas. Por ejemplo:
    
$$F(x, w, b) = b + w x$$
    
o bien:
    
$$ F(x,w,b) = 1.7159*tanh(w*x+b) $$
    
o bien:
    
$$ F(x,weights, biases) = Artificial-Neural-Network (ANN)$$
    
o bien:
    
$$ F(x,weights, biases) = Supported-Vector-Machine(SVM)$$
    
o bien: 
    
$$ F(x,weight, biases) = Decision-Tree $$
    
o bien:
    
$$ F(x,weight, biases) = Decision-Forest $$
    
entre otros.   

<font size=4 color='black'>

Dada la simplicidad de nuestros datos, para nuestro sistema de aprendizaje proponemos que la relación entre las vairables que describen nuestro sistema es una relación lineal, descrita por la función:
    
 $$F(x, w, b) = b + w x$$
    
En la nomenclatura de inteligencia artificial, esto significa que nuestro problema se resuelve con una "regresión lineal" (Linear regression).

<font size=5 color='blue'>

Estos son los datos que describen el movimiento de nuestro cuerpo

In [ ]:
desviacion = 10
beta = 4
m = 5000

x, y = generador_datos_simple(beta, m, desviacion)
plt.figure(figsize=(13,8))

plt.scatter(x, y)
plt.grid(True)
plt.title('Movimiento del cuerpo', size=24)
plt.xlabel('X', size=18)
plt.ylabel('Y', size=18);

<font size=4 color ='blue'>
Se generan histogramas de los las variables $x$ y $y$

In [ ]:
plt.figure(figsize=(13,5))

plt.subplot(1, 2, 1)
plt.grid(True)
plt.hist(x, bins=30, edgecolor='black', alpha=0.5)
plt.xlabel('X(s)', fontsize=16)
plt.ylabel('frecuencia', fontsize=16)

plt.subplot(1, 2, 2)
plt.grid(True)
plt.hist(y, bins=30, edgecolor='black', alpha=0.5)
plt.xlabel('Y(cm)', fontsize=16)
plt.ylabel('frecuencia', fontsize=16);

<font size=4, color='blue'>
El total de los datos son divididos en dos grupos: uno con el 90 % de los datos y el segundo con el restante 10 %

In [ ]:
#1. Los datos se cambian de posición aleatoriamente

from random import shuffle

c = list(zip(x, y))    # se juntan las muestras: x e y ---> (x,y)
shuffle(c)             # se cambia el orden de las muestras (x,y)
(x, y) = zip(*c)       # se separan las muestras: (x,y)---> x e y

print('Se tienen', len(x), 'muestras en total')

#2. Los datos se dividen

muestras_train = (x[0:int(0.90*len(x))], y[0:int(0.90*len(y))])
print('Se van a usar', len(muestras_train[0]), 'muestras para el ajuste')

muestras_test = (x[int(0.90*len(x)):], y[int(0.90*len(y)):])
print('Se van a usar', len(muestras_test[0]), 'muestras para probar')

#3. Se grafican ambos conjuntos

plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 1)
plt.grid(True)
plt.scatter(muestras_train[0], muestras_train[1])
plt.title('Muestras_train', size=20)
plt.xlabel('X(s)', size =15)
plt.ylabel('Y(cm)', size =15)

plt.subplot(1, 2, 2)
plt.grid(True)
plt.scatter(muestras_test[0], muestras_test[1], color='red')
plt.title('Muestras_test', size=20)
plt.xlabel('X(s)', size =15)
plt.ylabel('Y(cm)', size =15);

<font size=4, color='blue'>
   
Las muestras seleccionadas para hacer el entrenamiento se dividen en dos grupos:

El 90 % (este valor es solo un ejemplo) de ellos se emplea para hacer el entrenamiento (training set),
    
el 10 % restantes se emplean para evaluar el $\textbf {costo}$ que se obtiene en cada $\textbf {época}$ del ajuste (validation set).


<font size=4, color='black'>

Generamos la función "training" que contiene la arquitectura que emplearemos para entrenar el sistema aprendizaje. 
    
De momento, en esta función incluiremos también la métrica que emplearemos para obtener el entrenamiento, ejecutaremos el entrenamiento y generaremos las graficas del costo como función de la época. 
    
Estas tres últimas acciones normalmente se definen por separado.
    

In [ ]:
#Function to update weight and bias

def training(x_train, y_train, weight, bias, alpha, epochs, val_ratio=0.1):
    
    '''
    Esta función actualiza los parámetros (w,b) usando gradient descent
    Además separa a x e y en dos conjuntos: train y validation usando el val_ratio
    
    INPUT
        x,y: muestras
        weight: peso inicial
        bias: bias inicial
        alfa: learning rate
        epochs: int que define el numero de veces a actualizar a los parámetros
        val_ratio: porcentaje de los datos (x,y) a usar como conjunto de validación
    OUTPUT
        weights: lista con los pesos actualizados en cada iteración
        biases: lista con los bias actualizados en cada iteración
        costs: lista con los residuos actualizados en cada iteración para el conjunto de entrenamiento
        costs_val: lista con los residuos actualizados en cada iteración para el conjunto de validación'''
    
    #1. inicializacion de parametros
    
    x = np.squeeze(x_train)
    y = np.squeeze(y_train)
    alpha = alfa
    costs = 0
    d_w = 0.0
    d_b = 0.0
    m = len(x)

    #2.  Especificaciones de las graficas
    
    plt.figure(figsize=(13,8)) 
    plt.title('Cost vs epoch', size=24)
    plt.xlabel('epoch', size=18)
    plt.ylabel('Cost', size=18)
    
    #3. Separación de muestras en los conjuntos train y validation
    
    train_ratio = int((1.0-val_ratio)*len(x))  
   
    samples_train = (x[0:train_ratio], y[0:train_ratio])
    samples_val = (x[train_ratio:], y[train_ratio:])
    x = samples_train[0]
    y = samples_train[1]
    x_val = samples_val[0]
    y_val = samples_val[1]
    
    #3.1 Normalización
    
    mean_x = np.mean(x)
    std_x = np.std(x)
    
    mean_y = np.mean(y)
    std_y = np.std(y)
    
    x = (x-mean_x)/std_x
    y = (y-mean_y)/std_y
    x_val = (x_val-mean_x)/std_x
    y_val = (y_val-mean_y)/std_y
    
    #4. Calculo del costo por epoca para ambos conjuntos
       
    weights = []
    biases = []
    costs = []
    costs_val = []
    
    m_train = len(x)
    m_val = len(x_val)
    
    for i in range(epochs):

        # calculo de derivadas y el residuo
        cost = 0.0
        cost_val = 0.0
        
        for j in range(m_train):

            d_w += 2*(weight*x[j] + bias- y[j])*x[j]
            d_b += 2*(weight*x[j]+bias-y[j])
            cost += (y[j]-weight*x[j]-bias)**2

        cost /= m_train
        d_w /= m_train
        d_b /= m_train
        
        #calculo del costo de las muestras de validación
        
        for j in range(m_val):
            cost_val += (y_val[j]-weight*x_val[j]-bias)**2
        cost_val /= m_val
               
       
        weights.append(weight)      # se agregan los valores calculados a las listas
        biases.append(bias)
        costs.append(cost)
        costs_val.append(cost_val)
        
        #5. Actualizacion de los parametros

        weight = weight - alpha*d_w
        bias = bias - alpha*d_b
        
        plt.grid(True)
        plt.scatter(i, cost, color='blue')
        plt.scatter(i, cost_val, color='orange')
    plt.legend(['train', 'validation'],loc=0);

    return weights, biases, costs, costs_val, mean_x, std_x, mean_y, std_y

In [ ]:
x_train = muestras_train[0]
y_train = muestras_train[1]

In [ ]:
weight_0 = np.random.random()
bias_0 = np.random.random()
alpha = 0.04
num_epochs = 100
validation_ratio = 0.1

weights, biases, cost, cost_val, mean_x, std_x, mean_y, std_y = training \
        (x_train, y_train, weight_0, bias_0, alpha, num_epochs, validation_ratio)


<font size=5, color='blue'>

Inferencia

<font size=4, color='black'>

Dado el tiempo t el sistema de aprendizaje puede predecir (inferir) del valor de la distancia para ese tiempo.

In [ ]:
def inference(x, w, b, x_mean, x_std, y_mean, y_std) :
    
    x = (x-x_mean) / x_std 
    y = w * x + b
    y = y * y_std + y_mean
    
    return y

In [ ]:
tiempos = [1.65, 2.20, 4.5, 8.7]

for t in tiempos :
    distancia = inference(t,weights[-1], biases[-1], mean_x, std_x, mean_y, std_y)

    print ("Para el tiempo de {0:5.3f} s la distancia inferida es {1:6.3f} cm ".format(t,distancia))

In [ ]:
x_test = muestras_test[0]
y_test = muestras_test[1]

In [ ]:
x_test = np.squeeze(x_test)

In [ ]:
tiempos = x_test[:3]

for t in tiempos :
    distancia = inference(t,weights[-1], biases[-1], mean_x, std_x, mean_y, std_y)

    print ("Para el tiempo de {0:5.3f} s la distancia inferida es {1:6.3f} cm ".format(t,distancia))